### Set Up Environment

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load the data

In [48]:
def readcsv(file):
    return pd.read_csv(file)

sales_data_df = readcsv('sales_data.csv')
sales_data_df.head()

,transaction_id,customer_id,date,product_id,quantity,price
0,1,101,2023-01-01,201,2,19.99
1,2,102,2023-01-02,202,1,9.99
2,3,103,2023-01-03,203,5,4.99
3,4,104,2023-01-04,204,3,14.99
4,5,105,2023-01-05,201,1,19.99


### Data Cleaning

In [10]:
def data_cleaning(df):

    # Drop rows with missing values and create new dataframe of just missing value rows
    df_cleaned = df.dropna()
    df_missing = df[~df.index.isin(df_cleaned.index)]

    # Convert columns to appropriate data types
    df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])
    df_cleaned['price'] = pd.to_numeric(df_cleaned['price'])
    df_cleaned['quantity'] = pd.to_numeric(df_cleaned['quantity'])
    df_cleaned['customer_id'] = df_cleaned['customer_id'].astype('int')
    df_cleaned['product_id'] = df_cleaned['product_id'].astype('int')
    df_cleaned['transaction_id'] = df_cleaned['transaction_id'].astype('int')

    

In [49]:
# remove rows with missing values from sales_data_df and add rows with missing values to a new dataframe
sales_data_cleaned = sales_data_df.dropna()
sales_data_missing = sales_data_df[~sales_data_df.index.isin(sales_data_cleaned.index)]
sales_data_cleaned.size, sales_data_missing.size, sales_data_df.size

(180, 0, 180)

In [14]:
#Now, you can deal with the columns that have missing values
for col in missing_values_cols.index:
    # Here, you can add your code to deal with the missing values in each column.
    # For example, you might want to fill the missing values with the mean of the column:
    sales_data_df[col].fillna(sales_data_df[col].mean(), inplace=True)

In [15]:
sales_data_df.tail()

,transaction_id,customer_id,date,product_id,quantity,price
25,26,126,2023-03-06,214,4,9.990000
26,27,127,2023-03-07,205,2,29.990000
27,28,128,2023-03-08,206,1,39.990000
28,29,129,2023-03-09,207,3,24.990000
29,30,130,2023-03-10,203,2,19.852069


In [16]:
sales_data_df.isna().sum()

transaction_id    0
customer_id       0
date              0
product_id        0
quantity          0
price             0
dtype: int64